# 最常用的Chain

In [ ]:
# LLMChain 最常用的链式
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(model_name="gpt-3.5", temprature=0)
prompt_template = "帮我给{product}想三个注册的域名？"

chain = LLMChain(llm=llm, prompt=PromptTemplate(temprature=prompt_template,verbose=True))

chain({"product": "apple"})


### 顺序链 SimpleSequentialChain
- 将前一个LLM的输出作为下一个LLM的输入

In [24]:
# 只支持固定链路
from langchain.chains import SimpleSequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain.chains  import LLMChain

chat_model  = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# chain1
first_prompt = ChatPromptTemplate.from_template("帮我给{product}的公司起一个响亮容易记忆的名字?")

chain_one = LLMChain(llm=chat_model, prompt=first_prompt,verbose=True)

#chain2  
second_prompt = ChatPromptTemplate.from_template("用5个词来描述一下这个公司的名字:{company_name}")
                  
chain_two = LLMChain(llm=chat_model, prompt=second_prompt, verbose=True)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)


overall_simple_chain.run("农业")



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 帮我给农业的公司起一个响亮容易记忆的名字?

> Finished chain.
农耕丰收集团


> Entering new LLMChain chain...
Prompt after formatting:
Human: 用5个词来描述一下这个公司的名字:农耕丰收集团

> Finished chain.
农业、丰收、集团、耕种、农耕

> Finished chain.


'农业、丰收、集团、耕种、农耕'

# SequentialChain 支持多个链路的顺序执行



In [33]:

# SequentialChain 支持多个链路的顺序执行
from langchain.chains import SequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain.chains  import LLMChain

chat_model  = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

#chain 1 任务：翻译成中文

first_prompt = ChatPromptTemplate.from_template("把下面的内容翻译成中文：\n\n{content}")
chain_one = LLMChain(llm=chat_model, prompt=first_prompt, verbose=True, output_key="Chinese_Rview")

#chain 2 任务：对翻译后的中文进行总结摘要， input

second_prompt = ChatPromptTemplate.from_template("用一句话总结下面的内容：\n\n{Chinese_Rview}")
chain_two = LLMChain(llm=chat_model, prompt=second_prompt, verbose=True, output_key="Chinese_Summary")

#chain 3 任务: 智能识别语言， input_key 是上一个chain的output_key
third_prompt = ChatPromptTemplate.from_template("请识别下面的内容的语言？\n\n{Chinese_Summary}")
chain_three = LLMChain(llm=chat_model, prompt=third_prompt, verbose=True, output_key="Language")

#chain 4 任务: 针对摘要使用指定语言进行评论 input_key 是上一个chain的output_key
fourth_prompt = ChatPromptTemplate.from_template("用{Language}语言评论下面的摘要：\n\n{Chinese_Summary}")
chain_four = LLMChain(llm=chat_model, prompt=fourth_prompt, verbose=True, output_key="Comment")

#chain 5 任务: 对评论内容写个代码样列
fourth_prompt = ChatPromptTemplate.from_template("对下面评论内容写个代码样列：\n\n{Comment}")
chain_five = LLMChain(llm=chat_model, prompt=fourth_prompt, verbose=True, output_key="Code")
                     
#overall 任务: 翻译成中文->对翻译后的中文进行总结摘要-》智能识别语言-> 针对摘要使用指定语言进行评论

overall_sequential_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four,chain_five], verbose=True,
    input_variables=["content"],
    output_variables=["Chinese_Rview", "Chinese_Summary", "Language", "Comment", "Code"]
)

content_en = """
Both llm and chat_model are objects that represent configuration for a particular model. 
You can initialize them with parameters like temperature and others, and pass them around. 
The main difference between them is their input and output schemas. The LLM objects take string as input and output string. 
The ChatModel objects take a list of messages as input and output a message
"""
overall_sequential_chain(content_en)

# 并行链 ParallelChain







> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 把下面的内容翻译成中文：


Both llm and chat_model are objects that represent configuration for a particular model. 
You can initialize them with parameters like temperature and others, and pass them around. 
The main difference between them is their input and output schemas. The LLM objects take string as input and output string. 
The ChatModel objects take a list of messages as input and output a message


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 用一句话总结下面的内容：

llm和chat_model都是代表特定模型配置的对象。您可以使用参数（如温度等）对它们进行初始化，并将它们传递给其他地方。它们之间的主要区别在于它们的输入和输出模式。LLM对象以字符串作为输入和输出字符串。ChatModel对象以消息列表作为输入，并输出一条消息。

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 请识别下面的内容的语言？

llm和chat_model是代表特定模型配置的对象，具有不同的输入和输出模式。

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 用这段内容是中文。语言评论下面的摘要：

llm和chat_m

{'content': '\nBoth llm and chat_model are objects that represent configuration for a particular model. \nYou can initialize them with parameters like temperature and others, and pass them around. \nThe main difference between them is their input and output schemas. The LLM objects take string as input and output string. \nThe ChatModel objects take a list of messages as input and output a message\n',
 'Chinese_Rview': 'llm和chat_model都是代表特定模型配置的对象。您可以使用参数（如温度等）对它们进行初始化，并将它们传递给其他地方。它们之间的主要区别在于它们的输入和输出模式。LLM对象以字符串作为输入和输出字符串。ChatModel对象以消息列表作为输入，并输出一条消息。',
 'Chinese_Summary': 'llm和chat_model是代表特定模型配置的对象，具有不同的输入和输出模式。',
 'Language': '这段内容是中文。',
 'Comment': '这段内容描述了llm和chat_model这两个对象代表了特定模型配置，它们具有不同的输入和输出模式。',
 'Code': '```python\nclass llm:\n    def __init__(self, input_mode):\n        self.input_mode = input_mode\n\nclass chat_model:\n    def __init__(self, output_mode):\n        self.output_mode = output_mode\n\n# 创建llm对象和chat_model对象\nllm_obj = llm("input_mode1")\nchat_model_obj = chat